In [1]:
!nvidia-smi

Sun Jan 23 19:00:36 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P8    30W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install transformers
!pip install simpletransformers
!pip install wandb

     |████████████████████████████████| 3.4 MB 4.4 MB/s 
     |████████████████████████████████| 67 kB 5.3 MB/s 
     |████████████████████████████████| 3.3 MB 35.9 MB/s 
     |████████████████████████████████| 596 kB 52.7 MB/s 
     |████████████████████████████████| 895 kB 51.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 248 kB 4.1 MB/s 
     |████████████████████████████████| 311 kB 36.7 MB/s 
     |████████████████████████████████| 1.7 MB 41.6 MB/s 
     |████████████████████████████████| 9.3 MB 36.2 MB/s 
     |████████████████████████████████| 1.2 MB 44.6 MB/s 
     |████████████████████████████████| 43 kB 1.9 MB/s 
     |████████████████████████████████| 142 kB 54.9 MB/s 
     |████████████████████████████████| 180 kB 51.9 MB/s 
     |████████████████████████████████| 97 kB 6.4 MB/s 
     |████████████████████████████████| 63 kB 1

In [3]:
import pandas as pd
import numpy as np
from simpletransformers.classification import ClassificationModel
from sklearn.preprocessing import LabelEncoder

In [4]:
data = pd.read_table('/content/B3DB_classification.tsv')
X = data['SMILES']
target=data['BBB+/BBB-'].values.tolist()
label_encoder = LabelEncoder()
y_classification = label_encoder.fit_transform(target)

In [5]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(X, y_classification, test_size = 0.2, random_state = 42)
xtrain, xvalid, ytrain, yvalid = train_test_split(xtrain, ytrain, test_size = 0.16, random_state = 42)

In [6]:
Xtrain = {'text':xtrain, 'labels':ytrain}
Xvalid = {'text':xvalid, 'labels':yvalid}
Xtest = {'text':xtest, 'labels':ytest}
train_df = pd.DataFrame(Xtrain)
valid_df = pd.DataFrame(Xvalid)
test_df = pd.DataFrame(Xtest)

In [7]:
model = ClassificationModel('roberta', 'DeepChem/ChemBERTa-77M-MLM', args = {
'evaluate_during_training' : True,
'evaluate_during_training_verbose' : True,
'evaluate_during_training_steps' : 82,
'use_cached_eval_features ':True,
'use_early_stopping': True,
'early_stopping_delta': 0.01,
'early_stopping_metric':"auroc",
'early_stopping_metric_minimize' : False,
'early_stopping_patience' : 15,
'max_seq_length' : 128,
'num_train_epochs' : 20,
'train_batch_size' : 32,
'learning_rate' : 4e-5,
'reprocess_input_data': True, 
'overwrite_output_dir': True,
'auto_weights': True
}, num_labels=2,) # You can set class weights by using the optional weight argument


Downloading:   0%|          | 0.00/631 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.1M [00:00<?, ?B/s]

Some weights of the model checkpoint at DeepChem/ChemBERTa-77M-MLM were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.decoder.bias', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at DeepChem/ChemBERTa-77M-MLM and are newly initialized: ['classifier.out_proj.weight', 'classifier.den

Downloading:   0%|          | 0.00/6.80k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.07k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

In [8]:
import logging

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)


In [9]:
out = model.train_model(train_df, eval_df=valid_df,args={'wandb_project': 'bbb_model'})

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/5245 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_roberta_128_2_2


Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Initializing WandB run for training.


...

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/1000 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_roberta_128_2_2
INFO:simpletransformers.classification.classification_model:{'mcc': 0.7564212596391664, 'tp': 309, 'tn': 578, 'fp': 59, 'fn': 54, 'auroc': 0.9506532428610351, 'auprc': 0.9190893909846135, 'eval_loss': 0.2988056812286377}
INFO:simpletransformers.classification.classification_model: No improvement in auroc
INFO:simpletransformers.classification.classification_model: Current step: 15
INFO:simpletransformers.classification.classification_model: Early stopping patience: 15
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/1000 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_roberta_128_2_2
INFO:simpletransformers.classification.classification_model:{'mcc': 0.7564212596391664, 'tp': 309, 'tn': 578, 'fp': 59, 'fn': 54, 'auroc': 0.950847853445256, 'auprc': 0.9191914943773863, 'eval_loss': 0.2990507831573486}
INFO:simpletransformers.classification.classification_model: Patience of 15 steps reached
INFO:simpletransformers.classification.classification_model: Training terminated.
INFO:simpletransformers.classification.classification_model: Training of roberta model complete. Saved to outputs/.


In [10]:

model = ClassificationModel(
    "roberta", "/content/outputs/best_model",args={'wandb_project': 'bbb_model'}
)

In [11]:
import sklearn
result, model_outputs, wrong_predictions = model.eval_model(test_df,acc=sklearn.metrics.accuracy_score)
print(result)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/1562 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_roberta_128_2_2


Running Evaluation:   0%|          | 0/196 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Initializing WandB run for evaluation.


Training loss,█▇▅▅▅▄▃▆▄▄▅▄▃▅▄▃▅▄▃▂▂▃▂▁▅▃▃▄▃▆▄▂▃▄▁▂▂▂▁▄
auprc,▁▆▆▇▇▇▇▇████████████████████████████████
auroc,▁▅▅▇▇▇▇▇▇███████████████████████████████
eval_loss,█▆▆▃▃▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
fn,█▆▆▂▂▂▂▂▂▂▂▁▂▂▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
fp,▁▁▁██▆▆▆▆▇▆█▆▆██▇▇██▆▆▇▇▅▆▇▇▆▅▆▆▇▆▆▆▆▆▆▆
global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
lr,▂▄████▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁
mcc,▁▅▅▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇██████████████████████
tn,███▁▁▃▃▃▃▂▃▁▃▃▁▁▂▂▁▁▃▃▂▂▄▃▂▂▃▄▃▃▂▃▃▃▃▃▃▃
tp,▁▃▃▇▇▇▇▇▇▇▇█▇▇██▇▇██████████████████████


INFO:simpletransformers.classification.classification_model:{'mcc': 0.6780377249248517, 'tp': 460, 'tn': 867, 'fp': 134, 'fn': 101, 'auroc': 0.9261398138403487, 'auprc': 0.8803299644038921, 'acc': 0.8495518565941101, 'eval_loss': 0.36502233451726485}


{'mcc': 0.6780377249248517, 'tp': 460, 'tn': 867, 'fp': 134, 'fn': 101, 'auroc': 0.9261398138403487, 'auprc': 0.8803299644038921, 'acc': 0.8495518565941101, 'eval_loss': 0.36502233451726485}


In [12]:
result, model_outputs, wrong_predictions = model.eval_model(valid_df,acc=sklearn.metrics.accuracy_score)
print(result)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/1000 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_roberta_128_2_2


Running Evaluation:   0%|          | 0/125 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Initializing WandB run for evaluation.


INFO:simpletransformers.classification.classification_model:{'mcc': 0.7328883124539375, 'tp': 309, 'tn': 566, 'fp': 71, 'fn': 54, 'auroc': 0.9484108964628446, 'auprc': 0.9144241728264868, 'acc': 0.875, 'eval_loss': 0.29843237495422364}


{'mcc': 0.7328883124539375, 'tp': 309, 'tn': 566, 'fp': 71, 'fn': 54, 'auroc': 0.9484108964628446, 'auprc': 0.9144241728264868, 'acc': 0.875, 'eval_loss': 0.29843237495422364}


In [13]:
!sudo apt-get install git-lfs
!git lfs install
!git config --global credential.helper store

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 2,129 kB of archives.
After this operation, 7,662 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 git-lfs amd64 2.3.4-1 [2,129 kB]
Fetched 2,129 kB in 3s (844 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package git-lfs.
(Reading database ... 155229 files and directories cur

In [14]:
!huggingface-cli login



        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        To login, `huggingface_hub` now requires a token generated from https://huggingface.co/settings/token.
        (Deprecated, will be removed in v0.3.0) To login with username and password instead, interrupt with Ctrl+C.
        
Token: 
Login successful
Your token has been saved to /root/.huggingface/token


In [15]:
from transformers import AutoModel
from transformers import AutoTokenizer

model = AutoModel.from_pretrained("/content/outputs/best_model")
model.push_to_hub("BBB_prediction_finetuned_classification")

tokenizer = AutoTokenizer.from_pretrained("/content/outputs/best_model")
tokenizer.push_to_hub("BBB_prediction_finetuned_classification")

Some weights of the model checkpoint at /content/outputs/best_model were not used when initializing RobertaModel: ['classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at /content/outputs/best_model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Cloning ht

Upload file pytorch_model.bin:   0%|          | 3.37k/13.1M [00:00<?, ?B/s]

To https://huggingface.co/Parsa/BBB_prediction_finetuned_classification
   ed51d73..9f7a83a  main -> main

   ed51d73..9f7a83a  main -> main

To https://huggingface.co/Parsa/BBB_prediction_finetuned_classification
   9f7a83a..ff708bc  main -> main

   9f7a83a..ff708bc  main -> main



'https://huggingface.co/Parsa/BBB_prediction_finetuned_classification/commit/ff708bc175d4a18ccd7a3395458853e31ce13f0f'